# Project - Aegea

## Imports and packages

In [ ]:
from numba import jit
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium

In [ ]:
df = pd.read_csv("./data_source/amostra_menor.csv", sep=";")

In [ ]:
df_c = df.copy()

## Functions

In [ ]:
coords = df_c[['LATITUDE', 'LONGITUDE']].values

In [ ]:
@jit(nopython=True)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Raio da Terra em km
    # Convertendo para radianos manualmente dentro da função
    lat1, lon1, lat2, lon2 = lat1 * np.pi / 180.0, lon1 * np.pi / 180.0, lat2 * np.pi / 180.0, lon2 * np.pi / 180.0
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# Função para calcular distâncias e encontrar o ponto mais próximo
@jit(nopython=True)
def calculate_distances(coords):
    n = coords.shape[0]
    distances = np.full((n, n), np.inf)
    for i in range(n):
        for j in range(n):
            if i != j:
                distances[i, j] = haversine(coords[i, 0], coords[i, 1], coords[j, 0], coords[j, 1])
    return distances

In [ ]:
# Função para extrair os vizinhos mais próximos de uma matriz de distâncias
@jit(nopython=True)
def create_tour(distances):
    n = distances.shape[0]
    visited = np.zeros(n, dtype=np.bool_)
    tour = [0]  # start at the first point
    visited[0] = True

    for _ in range(1, n):
        last = tour[-1]
        next_idx = np.argmin(distances[last, :] + 1e9 * visited)
        tour.append(next_idx)
        visited[next_idx] = True

    tour.append(tour[0])  # return to start to close the cycle
    return tour

## Data cleaning

In [ ]:
df_c.info()

In [ ]:
df_c.head()

## Plot map

In [ ]:
dist_matrix = calculate_distances(coords)
tour = create_tour(dist_matrix)

In [ ]:
# Cria um mapa com um ponto central. Os valores centrais aqui são as médias das latitudes e longitudes
mapa = folium.Map(location=[df_c['LATITUDE'].mean(), df_c['LONGITUDE'].mean()], zoom_start=12)

# Adicionando marcadores
for i, line in df_c.iterrows():
    folium.CircleMarker(
        location=[line['LATITUDE'], line['LONGITUDE']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        popup=f"Latitude: {line['LATITUDE']}\nLongitude: {line['LONGITUDE']}"
    ).add_to(mapa)

# Adding lines to form the tour
for i in range(len(tour) - 1):
    start_point = [df.iloc[tour[i]]['LATITUDE'], df.iloc[tour[i]]['LONGITUDE']]
    end_point = [df.iloc[tour[i + 1]]['LATITUDE'], df.iloc[tour[i + 1]]['LONGITUDE']]
    folium.PolyLine([start_point, end_point], color="red").add_to(mapa)


# Salvar o mapa em um arquivo HTML
mapa.save('./maps/nearest_neighbor_map.html')